Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [3]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [162]:
def get_text():
    path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
    with open(path) as f:
        text = f.read().lower()
    print('corpus length:', len(text))
    return text

def build_dict(text):
    chars = sorted(list(set(text)))
    print('total chars:', len(chars))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    return char_indices, indices_char, len(chars)

def vectorize(text, char_indices, indices_char, dict_size, maxlen = 40, step = 3):
    ''' cut the text in semi-redundant sequences of maxlen characters'''
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    print('nb sequences:', len(sentences))

    print('Vectorization...')
    x = np.zeros((len(sentences), maxlen, dict_size), dtype=np.bool)
    y = np.zeros((len(sentences), dict_size), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1        
    return x, y

In [169]:
def build_model(units, maxlen, dict_size):
    ''' build the model: a single LSTM '''
    print('Build model...')
    model = Sequential()
    model.add(LSTM(units=units, input_shape=(maxlen, dict_size)))
    model.add(Dense(dict_size))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
    return model

In [179]:
def sample(preds, temperature=1.0):
    ''' helper function to sample an index from a probability array '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [182]:
def generate_samples(model, text, maxlen, char_indices, indices_char, dict_size, sample_size = 40):
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(sample_size):
            x_pred = np.zeros((1, maxlen, dict_size))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
            
        print()

In [191]:
def train_and_sample(model, x, y, iterations, epochs, maxlen, char_indices, indices_char, dict_size, sample_size):
    ''' train the model, output generated text after each iteration '''
    for iteration in range(1, iterations):
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x, y, batch_size=128, epochs=epochs, verbose = 1)

        if iteration % int(iterations / 10) == 0:
            generate_samples(model, text, maxlen, char_indices, indices_char, dict_size, 50)

In [126]:
maxlen, step = 2, 1

In [150]:
text = get_text()

corpus length: 600901


In [177]:
text = text[0:5000]

In [159]:
char_indices, indices_char, dict_size = build_dict(text)

total chars: 42


In [163]:
x, y = vectorize(text, char_indices, indices_char, dict_size, maxlen, step)

nb sequences: 4998
Vectorization...


In [164]:
x.shape, y.shape

((4998, 2, 42), (4998, 42))

In [193]:
model = build_model(32, maxlen, dict_size)
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 32)                9600      
_________________________________________________________________
dense_13 (Dense)             (None, 42)                1386      
_________________________________________________________________
activation_13 (Activation)   (None, 42)                0         
Total params: 10,986
Trainable params: 10,986
Non-trainable params: 0
_________________________________________________________________


In [174]:
model.fit(x, y, batch_size=128, epochs=50, verbose = 1);

Epoch 1/50
4998/4998 [==============================] - 0s - loss: 3.0032     
Epoch 2/50
4998/4998 [==============================] - 0s - loss: 2.6058     
Epoch 3/50
4998/4998 [==============================] - 0s - loss: 2.4483     
Epoch 4/50
4998/4998 [==============================] - 0s - loss: 2.3759     
Epoch 5/50
4998/4998 [==============================] - 0s - loss: 2.3170     
Epoch 6/50
4998/4998 [==============================] - 0s - loss: 2.2775     
Epoch 7/50
4998/4998 [==============================] - 0s - loss: 2.2359     
Epoch 8/50
4998/4998 [==============================] - 0s - loss: 2.2068     
Epoch 9/50
4998/4998 [==============================] - 0s - loss: 2.1741     
Epoch 10/50
4998/4998 [==============================] - 0s - loss: 2.1509     
Epoch 11/50
4998/4998 [==============================] - 0s - loss: 2.1214     
Epoch 12/50
4998/4998 [==============================] - 0s - loss: 2.0991     
Epoch 13/50
4998/4998 [==========================

In [178]:
genearte_samples(model, text, maxlen, char_indices, indices_char, dict_size, 50)

----- diversity: 0.2
----- Generating with seed: "ug"
uggle the and the been of struthe stion of st the be
----- diversity: 0.5
----- Generating with seed: "ug"
uggle the and pers the sperseady phy ity have per th
----- diversity: 1.0
----- Generating with seed: "ug"
uggll pues and--all posocrow! bef): ders
anialateare
----- diversity: 1.2
----- Generating with seed: "ug"
uggmionibled form thave noluscityrutens,
agasoutends


In [192]:
train_and_sample(model, x, y, 10, 5, maxlen, char_indices, indices_char, dict_size, 50)

--------------------------------------------------
Iteration 1
Epoch 1/5
4998/4998 [==============================] - 0s - loss: 1.7368     
Epoch 2/5
4998/4998 [==============================] - 0s - loss: 1.7380     
Epoch 3/5
4998/4998 [==============================] - 0s - loss: 1.7320     
Epoch 4/5
4998/4998 [==============================] - 0s - loss: 1.7376     
Epoch 5/5
4998/4998 [==============================] - 0s - loss: 1.7295     
----- diversity: 0.2
----- Generating with seed: " s"
 stiment the the purous the the the the prearithe an
----- diversity: 0.5
----- Generating with seed: " s"
 strut perthe be that the phistand perto itherthenti
----- diversity: 1.0
----- Generating with seed: " s"
 sperto tow whilen of whumatialless, te
prout grtain
----- diversity: 1.2
----- Generating with seed: " s"
 scans--ined noresia,strutherly mounicedich, onstre-
--------------------------------------------------
Iteration 2
Epoch 1/5
4998/4998 [==============================] - 0